### Excel file manipulation

Importing the given excel file and then replacing missing with nan

In [353]:
# import pandas as pd
import numpy as np

# Read the xlsx file
df = pd.read_excel('lables.xlsx')

# Select only the first 10 rows of the dataframe
df = df.head(886)

# Replace all occurrences of 'x' with NaN
df.replace('x', np.NaN, inplace=True)

columns_to_format = ['Giorno', 'Mese', 'Anno', 'Altitudine']

# Drop the lines were an error occurs for know
for index, row in df.iterrows():
    try:
        if pd.notna(row[columns_to_format]): 
            row[columns_to_format] = pd.to_datetime(row[columns_to_format], errors='coerce').strftime('%-d-%-m')
    except ValueError:
        # print(f"Error: {row[columns_to_format]} at row {index}")
        # Drop the rows that are too wierd
        df.drop(index, inplace=True)
        
# Chnage the days where they where automatically formatted to dates for two different fields
df[columns_to_format] = df[columns_to_format].apply(lambda x: pd.to_datetime(x, errors='coerce').strftime('%-d-%-m') if (not pd.isna(x) and ' ' in str(x)) else x)

In [354]:
import json

# define metadata list
metadata_list = []

# loop through rows of dataframe
for index, row in df.iterrows():
    # Do it only for the ones who have null note and others to avoid wierd things
    if pd.isnull(row['Note']) and pd.isnull(row['Determinavit']) and pd.isnull(row['Legit']): 
       
        # Fill the NaN in the row wiht the empty string
        row = row.fillna(' ')
        
        # create dictionary with metadata for this row
        metadata_dict = {
            "Nome_verbatim": row['Nome_verbatim'],
            "Nome_floritaly": row['Nome_floritaly'],
            "Località di raccolta": row['Località di raccolta'],
            "Altitudine": row['Altitudine'],
            "Giorno": row['Giorno'],
            "Mese": row['Mese'],
            "Anno": row['Anno'],
        }
        # create dictionary with "file_name" and "text" keys
        metadata_list.append({
            "file_name": f"{str(row['ID']).zfill(5)}.jpg",
            "text": json.dumps(metadata_dict)
        })

# write jsonline file
with open('metadata.jsonl', 'w') as outfile:
    for entry in metadata_list:
        json.dump(entry, outfile)
        outfile.write('\n')